In [1]:
# 造轮子: Promise

class Promise:
    def __init__(self, function):
        self.status = 'pending'
        #（1）对象的状态不受外界影响。Promise对象代表一个异步操作，
        # 有三种状态：pending（进行中）、fulfilled（已成功）和 rejected（已失败

        self.__resolve_func_args = []
        self.__resolve_func_kwargs = {}
        
        self.__reject_func_args = []
        self.__reject_func_kwargs = {}
        
        def __resolve(*args, **kwargs):
            #print(f'__resolve get arguments: {args} {kwargs}')
            #print(f'status: pending => fulfilled')
            self.status = 'fulfilled'
            
            self.__resolve_func_args = args
            self.__resolve_func_kwargs = kwargs
            
        def __reject(*args, **kwargs):
            #print(f'__reject get arguments: {args} {kwargs}')
            #print(f'status: pending => rejected')
            self.status = 'rejected'
            
            self.__reject_func_args = args
            self.__reject_func_kwargs = kwargs
            
        # execute funtion as soon as Promise is maded
        function(__resolve, __reject)
        
    def then(self, resolve_func):
        if self.status == 'fulfilled':
            resolve_func(*self.__resolve_func_args, **self.__resolve_func_kwargs)
        return self
        
    def catch(self, reject_func):
        if self.status == 'rejected':
            reject_func(*self.__reject_func_args, **self.__reject_func_kwargs)
        return self

        

In [2]:
import time
import random

def my_func0(resolve_func, reject_func):
    print('my function start')
    time.sleep(1)
    if random.random() > 0.5:
        print('succeed')
        resolve_func({"data": "abcdefg"})
    else:
        print('error')
        reject_func("THIS IS ERROR MESSAGE")
    time.sleep(1)
    print('my function done')

In [3]:
# print(my_func(
#     lambda data_dict: print('data:', data_dict['data']), 
#     lambda error_msg: print('error:', error_msg)
# ))

In [4]:
promise = Promise(my_func0)

my function start
succeed
my function done


In [5]:
promise.then(
    lambda data_dict: print('data:', data_dict['data'])
).catch(
    lambda error_msg: print('error:', error_msg)
)

data: abcdefg


In [6]:
def my_func1(resolve_func, reject_func):
    if random.random() > 0.5:
        resolve_func()
    else:
        reject_func()

for i in range(10):
    Promise(my_func1).then(
        lambda: Promise(my_func1).then(
            lambda: print('#### succeed succeed')
        ).catch(
            lambda: print('#### succeed failed')
        )
    ).catch(
        lambda: Promise(my_func1).then(
            lambda: print('#### failed succeed')
        ).catch(
            lambda: print('#### failed failed')
        )
    )

#### succeed failed
#### failed failed
#### failed succeed
#### failed succeed
#### succeed failed
#### failed failed
#### failed succeed
#### failed failed
#### failed failed
#### succeed failed
